# System of First Order ODEs

# EXAMPLE 10.26
Find approximate values of $y$ and $z$ corresponding to $x = 0.1$, given that $y(0) = 2, z(0) = 1$ and $dy/dx = x + z, dz/dx = x – y^2$.

In [26]:
import numpy as np
from scipy.integrate import solve_ivp

def f(t, Y):
    y, z = Y
    return [t + z, t - y**2]

t_eval = [0.1]

sol = solve_ivp(f, [0, 0.1], [2, 1], method='RK45', t_eval=[0.1], rtol=1e-6, atol=1e-6)

print(f't = {sol.t[0]:.1f}, y = {sol.y[0][0]:.6f}, z = {sol.y[1][0]:.6f}')

t = 0.1, y = 2.084544, z = 0.586785


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# ===================================
# PINN for System of ODEs (Example 10.26)
# ===================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net_y = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )
        self.net_z = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        y = self.net_y(x)
        z = self.net_z(x)
        return torch.cat([y, z], dim=1)

model = PINN().to(device)

x_colloc = torch.linspace(0, 0.1, 200, dtype=torch.float32, device=device, requires_grad=True).reshape(-1, 1)

x0 = torch.tensor([[0.0]], dtype=torch.float32, device=device)
y0 = torch.tensor([[2.0]], dtype=torch.float32, device=device)
z0 = torch.tensor([[1.0]], dtype=torch.float32, device=device)

def pinn_loss():
    YZ = model(x_colloc)
    y = YZ[:, 0:1]
    z = YZ[:, 1:2]
    
    dy = torch.autograd.grad(y, x_colloc, grad_outputs=torch.ones_like(y), create_graph=True)[0]
    dz = torch.autograd.grad(z, x_colloc, grad_outputs=torch.ones_like(z), create_graph=True)[0]
    
    f1 = dy - (x_colloc + z)
    f2 = dz - (x_colloc - y**2)
    
    loss_pde = torch.mean(f1**2) + torch.mean(f2**2)
    
    YZ0 = model(x0)
    loss_ic = torch.mean((YZ0[0, 0:1] - y0)**2) + torch.mean((YZ0[0, 1:2] - z0)**2)
    
    return loss_pde + loss_ic

optimizer_adam = optim.Adam(model.parameters(), lr=5e-4)

for it in range(10000):
    optimizer_adam.zero_grad()
    loss = pinn_loss()
    loss.backward()
    optimizer_adam.step()
    if it % 1000 == 0:
        print(f'Adam Iter {it}, Loss {loss.item():.3e}')

optimizer_lbfgs = optim.LBFGS(model.parameters(), lr=1.0, max_iter=500, line_search_fn='strong_wolfe')

def closure():
    optimizer_lbfgs.zero_grad()
    loss = pinn_loss()
    loss.backward()
    return loss
optimizer_lbfgs.step(closure)

x_pinn = torch.tensor([[0.1]], dtype=torch.float32, device=device)
YZ_pinn = model(x_pinn).detach().cpu().numpy()
y_pinn, z_pinn = YZ_pinn[0, 0], YZ_pinn[0, 1]

print(f'\nPINN Solution at x=0.1:')
print(f'y = {y_pinn:.6f}, z = {z_pinn:.6f}')
print(f'SciPy Solution at x=0.1:')
print(f'y = {sol.y[0][0]:.6f}, z = {sol.y[1][0]:.6f}')

# ===================================
# Plot
# ===================================
x_plot = np.linspace(0, 0.1, 200).reshape(-1, 1)
x_plot_torch = torch.tensor(x_plot, dtype=torch.float32, device=device)
YZ_plot = model(x_plot_torch).detach().cpu().numpy()
y_plot = YZ_plot[:, 0]
z_plot = YZ_plot[:, 1]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(x_plot, y_plot, 'r--', label='PINN y(x)', linewidth=2)
plt.axvline(x=0.1, color='g', linestyle=':', alpha=0.5)
plt.scatter([0.1], [sol.y[0][0]], color='b', s=100, label='SciPy y(0.1)', zorder=5)
plt.xlabel('x')
plt.ylabel('y')
plt.title('PINN vs SciPy: y(x)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

plt.subplot(1, 2, 2)
plt.plot(x_plot, z_plot, 'r--', label='PINN z(x)', linewidth=2)
plt.axvline(x=0.1, color='g', linestyle=':', alpha=0.5)
plt.scatter([0.1], [sol.y[1][0]], color='b', s=100, label='SciPy z(0.1)', zorder=5)
plt.xlabel('x')
plt.ylabel('z')
plt.title('PINN vs SciPy: z(x)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# EXAMPLE 10.27 
Find an approximate solution of x and y at t = 0.1 of the simultaneous equations $dx/dt = xy + 2t, dy/dt = 2ty + x$ subject to the initial conditions $x = 1, y = – 1, t = 0$.

In [25]:
from scipy.integrate import solve_ivp

def f(t, Y):
    x, y = Y
    return [x*y + 2*t, 2*t*y + x]

t_eval = [0.1]

sol = solve_ivp(f, (0, 0.1), [1, -1], method='RK45', t_eval=t_eval, rtol=1e-6, atol=1e-6)

print(f't = {sol.t[0]:.1f}, x = {sol.y[0][0]:.6f}, y = {sol.y[1][0]:.6f}')

t = 0.1, x = 0.918643, y = -0.913771


# EXAMPLE 10.28
Solve the differential equations $dy/dx = 1 + xz, dz/dx = -xy$  for $x = 0.3$ using the fourth order Runge-Kuta method. Intial values are x = 0, y = 0, z = 1.

In [24]:
from scipy.integrate import solve_ivp

def f(t, Y):
    y, z = Y
    return [1 + t*z, -t*y]

sol = solve_ivp(f, [0, 0.3], [0, 1], method='RK45', t_eval=[0.3], rtol=1e-6, atol=1e-6)

print(f't = {sol.t[0]:.1f}, y = {sol.y[0][0]:.6f}, z = {sol.y[1][0]:.6f}')

t = 0.3, y = 0.344823, z = 0.989990


# EXAMPLE 10.29
Find the value of $y(1.1)$ and $y(1.2)$ from $y'' + y^2y' = x^3, y(1) = 1, y'(1) = 1$, using the Taylor series method.

In [23]:
from scipy.integrate import solve_ivp

def f(t, Y):
    y, z = Y
    dy = z
    dz = t**3 - y**2*z
    return [dy, dz]

sol = solve_ivp(f, [1, 1.2], [1, 1], method='RK45', t_eval=[1.1, 1.2], rtol=1e-6, atol=1e-6)

for ti, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f't = {ti:.1f}, y = {yi:.6f}, z = {zi:.6f}')

t = 1.1, y = 1.100179, z = 1.005475
t = 1.2, y = 1.201516, z = 1.023547


# EXAMPLE 10.30
Using the Runge-Kutta method, solve $y'' = xy'^2 – y^2$ for $x = 0.2$ correct to 4 decimal places. Initial conditions are $x = 0, y = 1, y' = 0$.

In [28]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = x*z**2 - y**2
    return [dy, dz]

sol = solve_ivp(f, [0, 0.2], [1, 0], method='RK45', t_eval=[0.2], rtol=1e-6, atol=1e-6)

print(f'x = {sol.t[0]}, y = {sol.y[0][0]:.6f}, z = {sol.y[1][0]:.6f}')

x = 0.2, y = 0.980148, z = -0.196969


# EXAMPLE 10.31
Given $y'' + xy' + y = 0, y(0) = 1, y'(0) = 0$, obtain $y$ for $x = 0, 0.1, 0.3$ by 
any method. Further, continue the solution by Milne’s method to calculate 
$y(0.4)$.

In [29]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = -y - x*z
    return [dy, dz]

sol = solve_ivp(f, [0, 0.4], [1, 0], method='RK45', t_eval=[0, 0.1, 0.2, 0.3, 0.4], rtol=1e-6, atol=1e-6)

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.0, y = 1.000000, z = 0.000000
x = 0.1, y = 0.995012, z = -0.099503
x = 0.2, y = 0.980199, z = -0.196039
x = 0.3, y = 0.955997, z = -0.286799
x = 0.4, y = 0.923116, z = -0.369246


# Exercises 10.6

1. Apply Picard’s method to find the third approximations to $y$ and $z$ for $\frac{dy}{dx}=z,\; \frac{dz}{dx}=x^3(y+z)$ with $y(0)=1,\; z(0)=\tfrac12$.

2. Using Taylor series, find $x$ and $y$ at $t=0.4$ for $\frac{dx}{dt}=x+y+t,\; \frac{d^2y}{dt^2}=x-t$ with $x(0)=0,\; y(0)=1,\; y'(0)=-1$.

3. Solve using 4th-order Taylor series for $x=0.1,0.2$: $\frac{dy}{dx}=xz,\; \frac{dz}{dy}=xy$ with $x=0,\; y=1$.

4. Using RK4, compute $y(0.1),z(0.1),y(0.2),z(0.2)$ for $y'=x+z,\; z'=x-y^2$ with $y(0)=0,\; z(0)=1$.

5. Using Picard’s method find the second approximation for $\left(\frac{dy}{dx}\right)^2 = x^2 + 3xy^2$ with $y(0)=1,\; y'(0)=2$.

6. Use Picard’s method to approximate $y$ at $x=0.1$ for $\frac{d^2y}{dx^2} + \left(\frac{dy}{dx}\right)^2 + 2x^2 = 0$ with $y(0)=0.1,\; y'(0)=0.5$.

7. Find $y(0.2)$ using Taylor series for $y'' + 3xy' - 6y = 0$ with $y(0)=1,\; y'(0)=0.1$.

8. Using RK4, solve $y'' = y + xy'$ with $y(0)=1,\; y'(0)=0$ to find $y(0.2)$ and $y'(0.2)$.

9. Using RK4, find $y(0.2)$ for $y'' - 2y' + 2y = e^{2t}\sin t$ with $y(0)=-0.4,\; y'(0)=-0.6$.

10. Using RK4, find $\theta(0.2)$ and $\theta'(0.2)$ for $\theta'' + \frac{g}{l}\sin\theta = 0$ with $l=98\text{ cm}, g=980\text{ cm/s}^2$ and $\theta(0)=0,\; \theta'(0)=4.472$.

11. Using RK4 (step size $h=0.02$), compute $v(0.02)$ and $v'(0.02)$ for $v'' + \frac{R}{L}v' + \frac{1}{LC}v = 0$ with $v(0)=v_0,\; v'(0)=0$, where $v_0=10\text{ V}, C=0.1\text{ F}, L=0.5\text{ H}, R=10\Omega$.

# 1

In [31]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = x**3*(y + z)
    return [dy, dz]

sol = solve_ivp(f, [0, 0.1], [1, 1/2], method='RK45', t_eval=[0.1], rtol=1e-6, atol=1e-6)

print(f'x = {sol.t[0]}, y = {sol.y[0][0]:.6f}, z = {sol.y[1][0]:.6f}')

x = 0.1, y = 1.050001, z = 0.500039


# 2

In [38]:
from scipy.integrate import solve_ivp

def f(t, Y):
    x, y, z = Y
    dx = x + y + t
    dy = z
    dz = x - t
    return [dx, dy, dz]

sol = solve_ivp(f, [0, 0.4], [0, 1, -1], method='RK45', t_eval=[0, 0.1, 0.2, 0.3, 0.4], rtol=1e-6, atol=1e-6)

for ti, xi, yi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f't = {ti:.1f}, x = {xi:.6f}, y = {yi:.6f}')

t = 0.0, x = 0.000000, y = 1.000000
t = 0.1, x = 0.105171, y = 0.900004
t = 0.2, x = 0.221406, y = 0.800069
t = 0.3, x = 0.349881, y = 0.700359
t = 0.4, x = 0.491923, y = 0.601158


# 3

In [1]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = x*z + 1
    dz = x*y*(x*z + 1)
    return [dy, dz]

sol = solve_ivp(f, [0, 0.2], [1, 1], method='RK45', t_eval=[0, 0.1, 0.2], rtol=1e-6, atol=1e-6)

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.0, y = 1.000000, z = 1.000000
x = 0.1, y = 1.105014, z = 1.005707
x = 0.2, y = 1.220247, z = 1.026014


# 4

In [2]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = x + z
    dz = x - y**2
    return [dy, dz]

sol = solve_ivp(f, [0, 0.2], [0, 1], method='RK45', t_eval=[0, 0.1, 0.2])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.0, y = 0.000000, z = 1.000000
x = 0.1, y = 0.105158, z = 1.004641
x = 0.2, y = 0.221183, z = 1.016896


# 5

In [4]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = x**3*z + x**3*y
    return [dy, dz]

sol = solve_ivp(f, [0, 0.1], [1, 1/2], method='RK45', t_eval=[0.1])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.1, y = 1.050001, z = 0.500039


# 6

In [5]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = (-2*x*z - y) / (1 + 2*x)
    return [dy, dz]
sol = solve_ivp(f, [0, 0.1], [0.5, 0.1], method='RK45', t_eval=[0.1])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.1, y = 0.507615, z = 0.053439


# 7

In [6]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = 6*y - 3*x*z
    return [dy, dz]

sol = solve_ivp(f, [0, 0.2], [1, 0.1], method='RK45', t_eval=[0.2])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.2, y = 1.140400, z = 1.305935


# 8

In [7]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = y + x*z
    return [dy, dz]

sol = solve_ivp(f, [0, 0.2], [1, 0], method='RK45', t_eval=[0.2])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.2, y = 1.020201, z = 0.204040


# 9

In [9]:
from scipy.integrate import solve_ivp
import numpy as np

def f(x, Y):
    y, z = Y
    dy = z
    dz = 2*z - 2*y + np.exp(2*x)*np.sin(x)
    return [dy, dz]

sol = solve_ivp(f, [0, 0.2], [-0.4, -0.6], method='RK45', t_eval=[0.2])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.2, y = -0.525559, z = -0.640149


# 10

In [10]:
from scipy.integrate import solve_ivp
import numpy as np

def f(x, Y):
    y, z = Y
    dy = z
    dz = -10*np.sin(y)
    return [dy, dz]

sol = solve_ivp(f, [0, 0.2], [0, 4.472], method='RK45', t_eval=[0.2])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.2, y = 0.838071, z = 3.657185


# 11

In [13]:
from scipy.integrate import solve_ivp

def f(x, Y):
    y, z = Y
    dy = z
    dz = 20*(-z - y)
    return [dy, dz]
sol = solve_ivp(f, [0, 0.02], [10, 0], method='RK45', t_eval=[0.02])

for xi, yi, zi in zip(sol.t, sol.y[0], sol.y[1]):
    print(f'x = {xi:.1f}, y = {yi:.6f}, z = {zi:.6f}')

x = 0.0, y = 9.964863, z = -3.292416
